# Baseline: Обучение Multi-Branch MLP на размеченных данных


In [1]:
import os
import sys
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight

from model import MultiBranchMLP
from data_module import DataModule
from lightning_module import BaseLightningModule

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    import random
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


## 1. Загрузка данных


In [2]:
data_dir = '../data'

# --- Нормализация входов (минимальная правка baseline) ---
import data_module as dm_module

train_path = os.path.join(data_dir, 'train_labeled.csv')
train_df = pd.read_csv(train_path)

X_train = train_df.drop('target', axis=1).values.astype(np.float32) / 255.0
MEAN_T = torch.from_numpy(X_train.mean(axis=0)).float()
STD_T = torch.from_numpy(X_train.std(axis=0) + 1e-6).float()

# Патчим CSVDataset так, чтобы DataModule автоматически отдавал нормализованные тензоры
_BaseCSVDataset = dm_module.CSVDataset

class CSVDatasetNormalized(_BaseCSVDataset):
    def __getitem__(self, idx):
        x = torch.from_numpy(self.X[idx]).float() / 255.0
        x = (x - MEAN_T) / STD_T
        if self.has_target:
            y = int(self.y[idx])
            return x, y
        return x

dm_module.CSVDataset = CSVDatasetNormalized
# --- конец блока нормализации ---

dm = DataModule(
    data_dir=data_dir,
    batch_size=128,
    num_workers=0
)

dm.setup()

print(f'Input dimension: {dm.input_dim}')
print(f'Number of classes: {dm.n_classes}')
print(f'Labeled train samples: {len(dm.train_labeled_dataset)}')
print(f'Test samples: {len(dm.test_dataset)}')


Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000
Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000


## 2. Анализ дисбаланса классов и вычисление весов


In [3]:
train_labels = dm.train_labeled_dataset.y

unique_labels = np.unique(train_labels)
class_weights = compute_class_weight(
    'balanced',
    classes=unique_labels,
    y=train_labels
)

print(f'Class weights: {dict(zip(unique_labels, class_weights))}')

class_weights_tensor = torch.FloatTensor(class_weights)


Class weights: {0: 1.0738255033557047, 1: 0.963855421686747, 2: 1.0256410256410255, 3: 1.103448275862069, 4: 0.9523809523809523, 5: 0.935672514619883, 6: 0.9523809523809523, 7: 1.0596026490066226, 8: 0.9248554913294798, 9: 1.0457516339869282}


## 3. Создание модели


In [4]:
model = MultiBranchMLP(
    input_dim=dm.input_dim,
    hidden_dim=256,
    output_dim=dm.n_classes,
    num_blocks=4,
    dropout=0.1,
    combine_mode='concat'
)

print(f'Model parameters: {sum(p.numel() for p in model.parameters()):,}')


Model parameters: 4,080,650


## 4. Создание Lightning модуля


In [5]:
loss_fn = nn.CrossEntropyLoss(weight=class_weights_tensor)

lightning_model = BaseLightningModule(
    model=model,
    loss_fn=loss_fn,
    optimizer_type='adamw',
    learning_rate=1e-3,
    task_type='multiclass'
)


## 5. Обучение модели


In [6]:
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='best_model-{epoch:02d}-{val_accuracy:.4f}',
    monitor='val_accuracy',
    mode='max',
    save_top_k=1,
    save_last=True
)

trainer = Trainer(
    max_epochs=100,
    callbacks=[checkpoint_callback],
    enable_checkpointing=True,
    logger=True,
    enable_progress_bar=True,
    enable_model_summary=True,
    accelerator='auto',
    devices='auto'
)

trainer.fit(lightning_model, dm)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
You are using a CUDA device ('NVIDIA GeForce RTX 5070 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type             | Params | Mode  | FLOPs
-------------------------------------------------------------
0 | model   | MultiBranchMLP   | 4.1 M  | train | 0    
1 | loss_fn | CrossEntropyLoss | 0      | train | 0    
2 | metrics | ModuleDict       | 0      | train | 0    
-------------------------------------------------------------
4.1 M     Trainable params
0         Non-trainable params
4.1 M     Total params
16.323    Total estimated model params size (MB)
70        Modules in train mode


Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000


Sanity Checking: |                                                                               | 0/? [00:00<…

Epoch 0: accuracy=0.0859, f1_macro=0.0837


Training: |                                                                                      | 0/? [00:00<…

...

Validation: |                                                                                    | 0/? [00:00<…

Epoch 98: accuracy=0.3346, f1_macro=0.3355


Validation: |                                                                                    | 0/? [00:00<…

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: accuracy=0.3346, f1_macro=0.3354


## 6. Оценка на тестовой выборке


In [7]:
best_model_path = checkpoint_callback.best_model_path
print(f'Loading best model from: {best_model_path}')

if best_model_path:
    best_model = BaseLightningModule.load_from_checkpoint(
        best_model_path,
        model=model,
        loss_fn=loss_fn,
        optimizer_type='adamw',
        learning_rate=1e-3,
        task_type='multiclass'
    )
else:
    best_model = lightning_model

test_results = trainer.test(best_model, dm)

print('\n=== Финальные результаты на тестовой выборке ===')
for key, value in test_results[0].items():
    print(f'{key}: {value:.4f}')


Loading best model from: C:\Users\СИВ\Desktop\МИСИС\3 семестр\Глубокое машинное обучение\GIT\dl2025\lesson7\homework\baseline\checkpoints\best_model-epoch=30-val_accuracy=0.3375.ckpt


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Input dimension: 3072
Number of classes: 10
Labeled train samples: 1600
Test samples: 4000


Testing: |                                                                                       | 0/? [00:00<…

Test results: accuracy=0.3630, f1_macro=0.3631
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.3630000054836273
      test_f1_macro         0.36310774087905884
        test_loss            5.019516468048096
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

=== Финальные результаты на тестовой выборке ===
test_loss: 5.0195
test_accuracy: 0.3630
test_f1_macro: 0.3631
